In [1]:
from sam_models import *
from sam_orm_queries import create_sam_engine, get_session, find_user_by_username, get_project_with_full_details

engine, SessionLocal = create_sam_engine()

hpc-reader:$SAM_DB_PASSWORD@sam-sql.ucar.edu/sam


In [2]:
with get_session(SessionLocal) as session:
    # Find a user
    user = find_user_by_username(session, 'benkirk')
    if user:
        print(f"Found user: {user.full_name}")
        print(f"Primary GID: {user.primary_gid}")
        print(f"Primary email: {user.primary_email}")
        print(f"All emails: {', '.join(user.all_emails)}")

        # Get detailed email info
        print("Detailed email information:")
        for email_info in user.get_emails_detailed():
            primary_marker = " (PRIMARY)" if email_info['is_primary'] else ""
            active_marker = "" if email_info['active'] else " (INACTIVE)"
            print(f"  - {email_info['email']}{primary_marker}{active_marker}")

        # Find projects
        print("Detailed project information:")
        for p in user.all_projects:
            label = "" if p.active else " ** INACTIVE **"
            print(f"  {p.projcode}, {p.title}{label}")


Found user: Benjamin Shelton Kirk
Primary GID: 1000
Primary email: benkirk@ucar.edu
All emails: benkirk@ucar.edu
Detailed email information:
  - benkirk@ucar.edu
Detailed project information:
  SCSG0001, CSG systems project
  NCGD0058, Software Engineering At Scale (SEAS) ** INACTIVE **
  UESM0011, 2022 CESM tutorial August 8-12 ** INACTIVE **
  UCIS0004, GPU Training Allocation for Workshop ** INACTIVE **


In [3]:
def project_details(project):
    if project:
        print(f"--- Project Details ---")
        print(f"Project: {project.projcode}")
        print(f"Title: {project.title}")
        print(f"Lead: {project.lead.full_name}")
        if project.admin and project.lead != project.admin:
           print(f"Admin: {project.admin.full_name}") 
        for d in project.directories:
            label = "" if d.is_currently_active else " ** INACTIVE **"
            print(f"  Directory: {d.directory_name}{label}")
        # Show allocations by resource
        print(f"Allocations by resource:")
        allocs_by_resource = project.get_all_allocations_by_resource()        
        for resource_name, alloc in allocs_by_resource.items():
            resource = Resource.get_by_name(session,resource_name)
            label = "" if resource.is_active else " ** INACTIVE RESOURCE??? **"
            print(f"  {resource_name:12}: {alloc.amount:,.0f} (expires {alloc.end_date.date()}){label}")

        # Show users on project
        print(f"Users:")
        for user in project.users:
            print(f"  {user.username:12} {user.display_name:30} <{user.primary_email}>")


In [4]:
with get_session(SessionLocal) as session:
    # Find a project
    project = get_project_with_full_details(session, 'SCSG0001')
    project_details(project)

--- Project Details ---
Project: SCSG0001
Title: CSG systems project
Lead: Benjamin Shelton Kirk
  Directory: /glade/p/CSG ** INACTIVE **
  Directory: /gpfs/csfs1/cisl/csg
  Directory: /glade/p/cisl/CSG
  Directory: /stratus/USS
Allocations by resource:
  GLADE       : 10 (expires 2026-09-30)
  Data_Access : 1 (expires 2026-09-30)
  HPC_Futures_Lab: 1 (expires 2026-09-30)
  Campaign_Store: 20 (expires 2026-09-30)
  Casper      : 50,000 (expires 2026-09-30)
  Stratus     : 100 (expires 2026-09-30)
  Casper GPU  : 10,000 (expires 2026-09-30)
  Derecho     : 100,000,000 (expires 2026-09-30)
  Derecho GPU : 1,000,000 (expires 2026-09-30)
  Gust        : 5,000 (expires 2026-09-30)
  Gust GPU    : 1,000 (expires 2026-09-30)
Users:
  benkirk      Ben Kirk                       <benkirk@ucar.edu>
  csgteam      Ben Kirk                       <None>
  rory         Rory Kelly                     <rory@ucar.edu>
  klefevre     Karla LeFevre                  <klefevre@ucar.edu>
  valent       Rich

In [5]:
from sam_orm_queries import get_projects_expiring_soon
with get_session(SessionLocal) as session:
    # Get expiring projects (simple) - all resources
    print("\n--- Projects Expiring Soon (30 days, all resources) ---")
    expiring = get_projects_expiring_soon(session, days=30)
    #expiring = list(set(expiring))
    print(f"Found {len(expiring)} allocations expiring")
    for proj, alloc, res_name, days in expiring:
        print(f"\n{proj.projcode} / {days} days remaining")
        project_details(proj)


--- Projects Expiring Soon (30 days, all resources) ---
Found 42 allocations expiring

UCLA0033 / 27 days remaining
--- Project Details ---
Project: UCLA0033
Title: Spring Land Surface and Subsurface Temperature Anomalies and Subsequent Downstream Late Spring-Summer Droughts/Floods in North America and East Asia
Lead: Yongkang Xue
  Directory: /gpfs/csfs1/univ/ucla0033
Allocations by resource:
  Campaign_Store: 30 (expires 2025-11-30)
  Casper      : 10,000 (expires 2025-11-30)
  Derecho     : 629,920 (expires 2025-11-30)
Users:
  zhijiong     Zhijiong Cao                   <zhijiong@ucar.edu>
  hlhuang      Huilin Huang                   <hlhuang@ucar.edu>
  yongkang     Yongkang Xue                   <yongkang@ucar.edu>
  hpmaths      Hara Presad Nayak              <hpmaths@ucar.edu>
  qianli       Qian Li                        <qianli@ucar.edu>

UGMU0031 / 27 days remaining
--- Project Details ---
Project: UGMU0031
Title: Impact of atmospheric diabatic heating in the tropical Indi

In [6]:
from sam_models_jobs_queries import *

projcode='SCSG0001'
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
resource='Casper'

with get_session(SessionLocal) as session:

    top_users = get_user_usage_on_project(        
                    session, projcode,
                    start_date,
                    end_date,
                    limit=5)
    for user in top_users:
        print(f"{user['username']:12}: {user['charges']:.2f}")
                
    usage = get_project_usage_summary(session,
                                      projcode,
                                      start_date,
                                      end_date,
                                      resource)
    #print(usage)
    print(f"Project {projcode} ran {usage['total_jobs']} and used {usage['total_core_hours']:2f} core hours")

    trend = get_daily_usage_trend(session,
                                  projcode,
                                  start_date,
                                  end_date,
                                  resource)
    for day in trend[:5]:
        print(f"{day['date']}: {day['jobs']} jobs, {day['charges']} charges")

benkirk     : 2120606.22
negins      : 1293099.06
csgteam     : 1174065.03
rory        : 1048022.02
bneuman     : 12598.27
Project SCSG0001 ran 24443 and used 34609.788424 core hours
2024-01-01: 2 jobs, 0.00166667 charges
2024-01-02: 3 jobs, 2.00861101 charges
2024-01-03: 18 jobs, 2.91166674 charges
2024-01-04: 4 jobs, 0.01833333 charges
2024-01-05: 2 jobs, 0.00138889 charges


In [7]:
with get_session(SessionLocal) as session:
    for projcode in [ 'CESM0002', 'CESM0028', 'P93300065' ]:
        # Find a project
        project = get_project_with_full_details(session, projcode)
        project_details(project)    

        ancestors = project.get_ancestors()
        print(f"Ancestors: {[p.projcode for p in ancestors]}")

        # Navigate down the tree
        descendants = project.get_descendants()
        print(f"All descendants: {len(descendants)}")
        print(project.print_tree())

--- Project Details ---
Project: CESM0002
Title: CESM Nov 2024-April 2026
Lead: David M Lawrence
  Directory: /glade/p/cesm
  Directory: /glade/p/cesmLE ** INACTIVE **
  Directory: /glade/p/cesmLME ** INACTIVE **
  Directory: /gpfs/csfs1/cesm
  Directory: /gpfs/csfs1/collections/cmip/CMIP6
Allocations by resource:
  GLADE       : 1,000 (expires 2026-04-30)
  Campaign_Store: 18,492 (expires 2026-04-30)
  Casper      : 1,500,000 (expires 2026-04-30)
  Casper GPU  : 50,000 (expires 2026-04-30)
  Derecho     : 687,750,016 (expires 2026-04-30)
  Derecho GPU : 435,000 (expires 2026-04-30)
Users:
  cesmdata     Jim Edwards                    <None>
  ault         Toby Ault                      <ault@ucar.edu>
  gokhan       Gokhan Danabasoglu             <gokhan@ucar.edu>
  mholland     Marika Holland                 <mholland@ucar.edu>
  bitz         Cecilia Bitz                   <bitz@ucar.edu>
  lamar        Jean-Francois Lamarque         <lamar@ucar.edu>
  mlevy        Michael Levy      